# Comprehensive Analysis of FCFS (First-Come First-Served) Scheduler

## 1  Overview  
This module implements the **First-Come First-Served (FCFS)** CPU–scheduling algorithm, the simplest non-preemptive discipline in operating-system design.  
Processes are executed strictly in order of **arrival time**; once a process obtains the CPU it runs to completion without interruption.

---

## 2  Reference Implementation  

    # fcfs.py
    from collections import deque
    from process import Process
    from utility_functions import sort_by_arrival, init_current_time, run_to_completion
    
    def fcfs_schedule(process_list):
        arrival_q   = deque(sort_by_arrival(process_list))
        schedule    = []
        current_time = init_current_time(process_list)
    
        while arrival_q:
            proc = arrival_q.popleft()
            start, finish = run_to_completion(proc, current_time)
            schedule.append({
                'pid':        proc.pid,
                'start':      start,
                'finish':     finish,
                'turnaround': proc.turnaround_time
            })
            current_time = finish
    
        return schedule

---

## 3  Process Class (Assumed Attributes)

| Attribute          | Description                              |
|--------------------|------------------------------------------|
| `pid`              | Unique process identifier                |
| `arrival_time`     | Time the process enters the ready queue  |
| `burst_time`       | Total CPU time required                  |
| `turnaround_time`  | `finish − arrival_time` (filled later)   |

---

## 4  Function Signature  

    def fcfs_schedule(process_list: list[Process]) -> list[dict]

* **Input**  : list of `Process` objects  
* **Output** : list of execution-record dictionaries  

---

## 5  Data Structures  

| Name         | Type                 | Purpose / Operations                         |
|--------------|----------------------|---------------------------------------------|
| `arrival_q`  | `collections.deque`  | FIFO queue of processes sorted by arrival   |
| `schedule`   | `list[dict]`         | Chronological execution history             |
| `current_time` | `int` / `float`    | Simulation clock, advanced per completion   |

Helper utilities:

* `sort_by_arrival(pl)`   → list sorted by `arrival_time`  
* `init_current_time(pl)` → earliest arrival (or `0`)  
* `run_to_completion(proc, now)` → returns `(start, finish)` **and** sets `proc.turnaround_time`

---

## 6  Algorithm Breakdown  

1. **Initialization**  
   * Sort processes by arrival, push into a `deque`.  
   * Set `current_time` to the minimum arrival time (or `0`).  

2. **Main Loop** (`while arrival_q`)  
   1. Pop the leftmost process (`proc`).  
   2. Call `run_to_completion(proc, current_time)`  
        * `start  = max(current_time, proc.arrival_time)`  
        * `finish = start + burst_time`  
        * `proc.turnaround_time = finish − arrival_time`  
   3. Append record to `schedule`.  
   4. `current_time = finish`.  

3. **Termination** – loop ends when all processes are executed.

---

## 7  Key Variables  

| Variable       | Role                                                                  |
|----------------|-----------------------------------------------------------------------|
| `current_time` | Tracks the instant the CPU becomes free for the next process          |
| `proc`         | Process currently being dispatched                                    |
| `start`/`finish` | Execution window of `proc`, returned by `run_to_completion`         |

---

## 8  Design Decisions  

| Decision                          | Rationale                                    |
|-----------------------------------|----------------------------------------------|
| `deque` for arrival queue         | O(1) pops from head; natural for FCFS        |
| Helper functions for utilities    | Improves readability and re-usability        |
| Fast-forward when CPU idle        | Avoids step-wise clock increments            |
| No ready-queue dictionary needed  | FCFS never re-orders tasks                   |

---

## 9  Edge Cases Handled  

1. **Idle gap before first arrival** – handled by `init_current_time` and `start=max(...)`.  
2. **Tied arrival times** – preserves original (stable) order.  
3. **Zero-burst process** – `finish == start`; turnaround equals waiting time.  
4. **Empty process list** – returns empty `schedule`.

---

## 10  Complexity Analysis  

* **Initialization** : `sort_by_arrival` → **O(n log n)**  
* **Main loop**    : one pop per process → **O(n)**  
* **Total time**   : **O(n log n)**  
* **Space**        : `arrival_q` + `schedule` → **O(n)**  





# Deep-Dive Documentation of `fcfs_schedule` — *What the Code **Really** Does and the Exact Road-Map it Follows*

Below is a **single-cell, self-contained guide** that not only describes the *“what”* of the FCFS implementation but also the *“how”*—the precise, ordered path the algorithm walks through at run-time.

---

## 1 Purpose

*Implement an **FCFS (First-Come First-Served)** scheduler that*  
1. **orders all processes by their arrival time**,  
2. **simulates CPU execution** in that order (no pre-emption), and  
3. **returns a chronologically accurate schedule** of when each process starts, finishes, and its overall turnaround time.

---

## 2 High-Level Pipeline (Road-Map)

| Step | Phase                                | Key Operation(s) & Code Lines | Output of the Step             |
|------|--------------------------------------|-------------------------------|--------------------------------|
| ➊    | **Pre-processing**                   | `sort_by_arrival` → line 6    | `arrival_q` = deque(sorted_processes) |
| ➋    | **Clock Initialisation**             | `init_current_time` → line 7  | `current_time` = earliest arrival (or 0) |
| ➌    | **Main Loop**                        | `while arrival_q:`            | Iteratively pops processes     |
| ➍    | &nbsp;&nbsp;&emsp;➊ **Dispatch**     | `proc = arrival_q.popleft()`  | Next process to run            |
| ➎    | &nbsp;&nbsp;&emsp;➋ **Run to Completion** | `run_to_completion(proc, current_time)` | (`start`, `finish`) timestamps *+* `proc.turnaround_time` |
| ➏    | &nbsp;&nbsp;&emsp;➌ **Record Entry** | Append dict to `schedule`     | `schedule` grows by 1          |
| ➐    | &nbsp;&nbsp;&emsp;➍ **Advance Clock** | `current_time = finish`       | Clock ready for next iteration |
| ➑    | **Termination**                      | `while` loop ends             | Full `schedule` list returned  |

> **Mind-set**:  
> *The algorithm is *event-driven*—time “jumps” from one process completion to the next; there is never a per-tick loop.*

---

## 3 Detailed Walk-Through With Comments

```python
# ─── Imports & Helpers ────────────────────────────────────────────────────
from collections import deque
from process import Process
from utility_functions import (
    sort_by_arrival,   # stable sort by arrival_time
    init_current_time, # returns min(arrival_time) or 0
    run_to_completion  # simulates proc until finish; returns (start, finish)
)

# ─── Core Function ────────────────────────────────────────────────────────
def fcfs_schedule(process_list):
    # 1) Build arrival-time FIFO queue  (Road-Map Step ➊)
    arrival_q = deque(sort_by_arrival(process_list))

    # 2) Initialise empty schedule + simulation clock (➋)
    schedule     = []
    current_time = init_current_time(process_list)

    # 3) Main simulation loop (➌)
    while arrival_q:
        # • Dispatch next process (➍)
        proc = arrival_q.popleft()

        # • Run it until it finishes (➎)
        start, finish = run_to_completion(proc, current_time)

        # • Persist execution record (➏)
        schedule.append({
            'pid'       : proc.pid,
            'start'     : start,
            'finish'    : finish,
            'turnaround': proc.turnaround_time
        })

        # • Advance the global clock (➐)
        current_time = finish

    # 4) All processes done  (➑)
    return schedule
